# **Pokémon Diffusion<a id="top"></a>**

> #### ``04-Training-Diffusion-Model.ipynb``

<i><small>**Alumno:** Alejandro Pequeño Lizcano<br>Última actualización: 11/03/2024</small></i></div>

TODO: INTRODUCIR MEJOR

### Training

Como paso final, se procede a entrenar el modelo de difusión. Para ello, se ha definido la función ``training()`` que engloba todo el proceso de difusión completo, tanto hacia adelante como hacia atrás y los ploteos de las muestras generadas. Para implementar el training hemos usado el **Algoritmo 1** de [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239) y se ha modificado para que sea capaz de generar imágenes condicionadas a una etiqueta.

<div style="text-align:center">
<img src="../figures/notebook_figures/algorithm1_training.png" width="40%" height="30%" />
</div>

Tambiñen se han añadido unas funcionalidades extra que permiten guardar cada epoch el modelo y sus pesos en un fichero con extensión .h5. Esto se hace para poder cargar el modelo y continuar el entrenamiento desde donde se quedó en caso de que se interrumpa por algún motivo.

<span style="color: red; font-size: 1.5em;">&#9888;</span> <i><small>**NOTA:** Por cada epoch se guarda en un fichero con extensión .h5 tanto el modelo como sus pesos. Este proceso se realiza ya que todo el entrenamiento es muy costoso y si se interrumpe por algún motivo, se puede volver a cargar el modelo y continuar el entrenamiento desde donde se quedó.

También cabe destacar que para una mayor eficiencia en el entrenamiento, se ha optado por realizar el ``sampling()`` cada 5 epochs.
</small></i>

In [ ]:
# Algorithm 1: Training
# =====================================================================
def training(
    model: tf.keras.models.Model,
    dataset: tf.data.Dataset,
    optimizer: tf.keras.optimizers.Optimizer,
    loss_fn: tf.keras.losses.Loss,
    total_epochs: int = 10,
    scheduler: str = "cosine",
    T: int = 100,
) -> None:
    """
    Performs the training loop.

    :param model: The model to train.
    :param dataset: The training dataset.
    :param optimizer: The optimizer to use.
    :param loss_fn: The loss function to use.
    :param total_epochs: The number of epochs to train for.
    :param scheduler: The type of schedule to use. Options are "linear" or "cosine".
    :param T: The number of timesteps to sample for.
    :return: None
    """

    # Save intermodels by epoch
    # =====================================================================

    # Create the folder to save the models by epoch
    folder_epoch = f"../../models/inter_models/diffusion_{IMG_SIZE}_{BATCH_SIZE}_{EPOCHS}_{T}_{scheduler}_ddpm"
    if not os.path.exists(folder_epoch):
        os.makedirs(folder_epoch)

    # Check if there are checkpoints to load
    if len(glob.glob(f"{folder_epoch}/diffusion_{scheduler}_*.h5")) > 0:

        last_checkpoint = sorted(
            glob.glob(f"{folder_epoch}/diffusion_{scheduler}*.h5"),
            key=lambda x: int(x.split("_")[-1].split(".")[0]),
        )[
            -1
        ]  # Get the last checkpoint
        print(f"Loading checkpoint {last_checkpoint}...")

        # Get the epoch from the checkpoint
        prev_epoch = int(
            last_checkpoint.split("_")[-1].split(".")[0]
        )  # Get the epoch from the checkpoint
        print(f"Resuming training from epoch {prev_epoch}...")

        model = tf.keras.models.load_model(
            f"{folder_epoch}/diffusion_{scheduler}_{prev_epoch}.h5"
        )  # Load the model

    else:
        prev_epoch = 0
        print("No checkpoints found, starting training from scratch...")

    # Start the training loop
    # =====================================================================

    # Get scheduler values
    beta = beta_scheduler(scheduler, T, beta_start, beta_end)  # Get beta
    alpha = 1.0 - beta  # Get alpha
    alpha_cumprod = np.cumprod(alpha)  # Get alpha cumulative product

    for epoch in trange(
        prev_epoch,
        total_epochs,
        desc=f"Training",
        total=total_epochs - prev_epoch,
        leave=True,
    ):  # 1: repeat (iterations through the epochs)
        for step, input_data in tqdm(
            enumerate(dataset),
            desc=f"Epoch {epoch+1}/{total_epochs}",
            total=len(dataset),
            leave=True,
        ):  # 1: repeat (iterations through the batches)
            # Generate a single timestep for one entire batch
            t = np.random.randint(0, T)
            normalized_t = np.full(
                (input_data.shape[0], 1), t / T, dtype=np.float32
            )  # 3: t ~ U(0, T)

            # Get the target noise
            noised_data = forward_diffusion(input_data, t, scheduler)  # 2: x_0 ~ q(x_0)
            target_noise = noised_data - input_data * np.sqrt(
                alpha_cumprod[t]
            )  # 4: eps_t ~ N(0, I)

            # 5: Take a gradient descent step on
            with tf.GradientTape() as tape:
                predicted_noise = model(
                    [noised_data, normalized_t], training=True
                )  # eps_theta -> model(x_t, t/T)
                loss = loss_fn(
                    target_noise, predicted_noise
                )  # gradient of the loss (MSE(eps_t, eps_theta))
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        print(f"EPOCH {epoch+1} LOSS: {loss.numpy():.4f} \n{'='*69}")

        # Save the model at the end of every 20 epochs
        if (epoch + 1) % 20 == 0:
            print(f"\tSaving model in epoch {epoch+1}")
            model.save(f"{folder_epoch}/diffusion_{scheduler}_{epoch+1}.h5")

        # Sample and plot every 10 epochs
        if (epoch + 1) % 10 == 0:
            print("\tSampling images...")
            plot_samples(model, num_samples=3, scheduler=scheduler, T=T)

In [ ]:
# Train the model
# =====================================================================
training(
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    loss_fn=loss_fn,
    scheduler="cosine",
    num_epochs=EPOCHS,
)

### Save Model

Finalmente, se guardan los resultados finales del modelo de difusión en un fichero `.h5` para su posterior uso y visualización. TODO: MIRAR OTROS FORMATOS DE GUARDADO

TODO: INVESTIGAR OTROS FORMATOS DE GUARDADO (HDF5, PICKLE, ETC.)

In [ ]:
# Save the model function
# =====================================================================
def save_model(model: tf.keras.models.Model, model_name: str) -> None:
    """Saves the model

    :param model: The model to save
    :param model_name: The name of the model
    :return: None
    """

    # Save the model
    model_dir = "./diffusion_models/models/"
    os.makedirs(model_dir, exist_ok=True)
    if not os.path.exists(os.path.join(model_dir, f"{model_name}.h5")):
        model.save(os.path.join(model_dir, f"{model_name}.h5"))
        print(f"Model {model_name}, saved successfully!")
    else:
        print(f"Model {model_name}, already exists!")

In [ ]:
# Save the model
model_name = f"diffusion_{IMG_SIZE}_{BATCH_SIZE}_{EPOCHS}_{T}_{scheduler}_ddpm"

save_model(model, model_name)

[BACK TO TOP](#top)